In [1]:
!pip install psycopg2

In [2]:
import pandas
import configparser
import psycopg2

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
db = config['myaws']['database']
user = config['myaws']['user']
pwd = config['myaws']['password']

In [4]:
conn = psycopg2.connect(host = host,
                       user = user,
                        password = pwd,
                        dbname = db
                       )
cur = conn.cursor()

In [5]:
table_sql = """
            CREATE TABLE IF NOT EXISTS indeed_gp5
            (
                id SERIAL,
                job_title VARCHAR(200),
                job_company VARCHAR(200),
                job_loc VARCHAR(200),
                job_salary VARCHAR(200),
                job_summary TEXT,
                PRIMARY KEY(id)
            );

            """

In [6]:
cur.execute(table_sql)
conn.commit()

# Request HTML

In [7]:
url = 'https://www.indeed.com/jobs?q=political+analyst&l=Washington%2C+DC&start=0'

In [8]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
print(html_data.decode('utf-8'))

<!DOCTYPE html>
<html lang="en" dir="ltr">
<head>
<meta http-equiv="content-type" content="text/html;charset=UTF-8">
<script type="text/javascript" src="/s/798993d/en_US.js"></script>
<link href="/s/60d7c47/jobsearch_all.css" rel="stylesheet" type="text/css">
<link rel="alternate" type="application/rss+xml" title="Political Analyst Jobs, Employment in Washington, DC" href="http://rss.indeed.com/rss?q=political+analyst&l=Washington%2C+DC">
<link rel="alternate" media="only screen and (max-width: 640px)" href="/m/jobs?q=political+analyst&l=Washington%2C+DC">
<link rel="alternate" media="handheld" href="/m/jobs?q=political+analyst&l=Washington%2C+DC">

<script type="text/javascript">
    
    if (typeof window['closureReadyCallbacks'] == 'undefined') {
        window['closureReadyCallbacks'] = [];
    }

    function call_when_jsall_loaded(cb) {
        if (window['closureReady']) {
            cb();
        } else {
            window['closureReadyCallbacks'].push(cb);
        }
    }
</

# Parese HTML

In [9]:
!pip install beautifulsoup4

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
print (soup)

<!DOCTYPE html>

<html dir="ltr" lang="en">
<head>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<script src="/s/798993d/en_US.js" type="text/javascript"></script>
<link href="/s/60d7c47/jobsearch_all.css" rel="stylesheet" type="text/css"/>
<link href="http://rss.indeed.com/rss?q=political+analyst&amp;l=Washington%2C+DC" rel="alternate" title="Political Analyst Jobs, Employment in Washington, DC" type="application/rss+xml"/>
<link href="/m/jobs?q=political+analyst&amp;l=Washington%2C+DC" media="only screen and (max-width: 640px)" rel="alternate"/>
<link href="/m/jobs?q=political+analyst&amp;l=Washington%2C+DC" media="handheld" rel="alternate"/>
<script type="text/javascript">
    
    if (typeof window['closureReadyCallbacks'] == 'undefined') {
        window['closureReadyCallbacks'] = [];
    }

    function call_when_jsall_loaded(cb) {
        if (window['closureReady']) {
            cb();
        } else {
            window['closureReadyCallbacks'].push(cb);
 

In [11]:
for table_resultsBody in soup.find_all('table', id = 'resultsBody'):
    pass
    #print(table_resultsBody)

In [12]:
for table_pageContent in table_resultsBody.find_all('table', id = 'pageContent'):
    pass
    #print(table_pageContent)

In [13]:
for td_resultsCol in table_pageContent.find_all('td', id = 'resultsCol'):
    pass
    #print(td_resultsCol)

# Save Data

In [14]:
#  identify the job title, company, ratings, reviews, salary, and summary
for div_row in td_resultsCol.find_all('div', class_='jobsearch-SerpJobCard unifiedRow row result'):

    # find job title
    job_title = None
    job_company  = None
    job_rating = None
    job_loc = None
    job_salary = None
    job_summary = None
    for div_title in div_row.find_all('div', class_ = 'title'):
        job_title = div_title.text.strip().replace("'","_")
    
    for div_dsc in div_row.find_all('div', class_ = 'sjcl'):
        
        #find company name
        for span_company in div_dsc.find_all('span', class_ = 'company'):
            job_company = span_company.text.strip().replace("'","_")
        
        # find location
        for div_loc in div_dsc.find_all('div', class_ = 'location accessible-contrast-color-location'):
            job_loc = div_loc.text.strip().replace("'","_")
    
    # find salary
    for div_salary in div_row.find_all('div',class_ ='salarySnippet'):
        job_salary = div_salary.text.strip().replace("'","_")
    
    #find summary
    for div_summary in div_row.find_all('div', class_ = 'summary'):
        job_summary = div_summary.text.strip().replace("'","_")
        
    # insert into database
    
    sql_insert = """
                insert into indeed_gp5(job_title,job_company,job_loc,job_salary,job_summary)
                values('{}','{}','{}','{}','{}')            
                """.format(job_title,job_company,job_loc,job_salary,job_summary)

    cur.execute(sql_insert)
    conn.commit()

In [15]:
df = pandas.read_sql_query('select * from indeed_gp5', conn)
df[:]

,id,job_title,job_company,job_loc,job_salary,job_summary
0,1,Intelligence Analyst,National Security Agency,"Fort Meade, MD","$47,016 - $74,759 a year",Intelligence Analysts have the power to influe...
1,2,Undergraduate Internship/Co-op Program - Intel...,Central Intelligence Agency,"Washington, DC",$21.21 - $22.43 an hour,"As an Intelligence Analyst Intern for the CIA,..."
2,3,Leadership Analyst,Central Intelligence Agency,"Washington, DC","$55,539 - $82,326 a year",As a Leadership Analyst for the CIA you will s...
3,4,Policy Analyst,The Groundwork Collaborative,"Washington, DC",None,As a Policy Analyst you’ll…*. The Policy Analy...
4,5,Data Analyst,The Groundwork Collaborative,"Washington, DC",None,As a Data Analyst you’ll…*. The Data Analyst w...
5,6,Political Analyst,Central Intelligence Agency,None,"$55,539 - $82,326 a year","As a Political Analyst for the CIA, you will s..."
6,7,Public Health Analyst,Government of the District of Columbia,None,"$64,603 - $83,287 a year",All qualified candidates will receive consider...
7,8,Policy Analyst,Service Employees International Union,None,"$65,273 a year",Experience with issue development in a politic...
8,9,Analyst,TDI,None,None,An Analyst is expected to comprehend complex g...
9,10,Program Analyst (Operations),Government of the District of Columbia,None,"$85,149 - $109,710 a year",All qualified candidates will receive consider...


In [16]:
cur.close()
conn.close()

# Extra Credit Wordle

In [17]:
for duty in df['job_summary']:
    print (type(duty))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [18]:
duties = ''
for duty in df['job_summary']:
    duties = duty +duties
print(duties)

As a Political Analyst for the CIA, you will support policymakers by producing and delivering written and oral assessments of the domestic politics, foreign…Mentoring, coaching, and leading other Language Analysts; In most cases, Experienced Level Language Analysts at NSA will have substantial experience in one or…Our research analysts produce detailed, comprehensive, and 100% accurate reports on the individuals (public figures, activists, legislators, political…The Situation Room, anchored by Wolf Blitzer, assembles top CNN correspondents, analysts, contributors and guests for complete, up-to-the minute coverage of the…Policy analysis, data science, political science, legislative or executive agency policymaking, or government affairs consulting.All qualified candidates will receive consideration without regard to race, color, religion, national origin, sex, age, marital status, personal appearance,…Support Budget Analysts and Team Lead with a variety of financial management and admin

In [19]:
f = open("file.txt", "w")
f.write(duties)
f.close() 

In [20]:
duties_file = open('file.txt','r').read()
for line in duties_file:
    print(line)

A
s
 
a
 
P
o
l
i
t
i
c
a
l
 
A
n
a
l
y
s
t
 
f
o
r
 
t
h
e
 
C
I
A
,
 
y
o
u
 
w
i
l
l
 
s
u
p
p
o
r
t
 
p
o
l
i
c
y
m
a
k
e
r
s
 
b
y
 
p
r
o
d
u
c
i
n
g
 
a
n
d
 
d
e
l
i
v
e
r
i
n
g
 
w
r
i
t
t
e
n
 
a
n
d
 
o
r
a
l
 
a
s
s
e
s
s
m
e
n
t
s
 
o
f
 
t
h
e
 
d
o
m
e
s
t
i
c
 
p
o
l
i
t
i
c
s
,
 
f
o
r
e
i
g
n
…
M
e
n
t
o
r
i
n
g
,
 
c
o
a
c
h
i
n
g
,
 
a
n
d
 
l
e
a
d
i
n
g
 
o
t
h
e
r
 
L
a
n
g
u
a
g
e
 
A
n
a
l
y
s
t
s
;
 
I
n
 
m
o
s
t
 
c
a
s
e
s
,
 
E
x
p
e
r
i
e
n
c
e
d
 
L
e
v
e
l
 
L
a
n
g
u
a
g
e
 
A
n
a
l
y
s
t
s
 
a
t
 
N
S
A
 
w
i
l
l
 
h
a
v
e
 
s
u
b
s
t
a
n
t
i
a
l
 
e
x
p
e
r
i
e
n
c
e
 
i
n
 
o
n
e
 
o
r
…
O
u
r
 
r
e
s
e
a
r
c
h
 
a
n
a
l
y
s
t
s
 
p
r
o
d
u
c
e
 
d
e
t
a
i
l
e
d
,
 
c
o
m
p
r
e
h
e
n
s
i
v
e
,
 
a
n
d
 
1
0
0
%
 
a
c
c
u
r
a
t
e
 
r
e
p
o
r
t
s
 
o
n
 
t
h
e
 
i
n
d
i
v
i
d
u
a
l
s
 
(
p
u
b
l
i
c
 
f
i
g
u
r
e
s
,
 
a
c
t
i
v
i
s
t
s
,
 
l
e
g
i
s
l
a
t
o
r
s
,
 
p
o
l
i
t
i
c
a
l
…
T
h
e
 
S
i
t
u
a
t
i
o
n
 
R
o
o
m
,
 
a
n
c
h
o
r
e
d
 
b
y
 
W


b
e
r
 
o
f
 
G
r
o
u
n
d
w
o
r
k
’
s
 
P
o
l
i
c
y
 
a
n
d
R
e
s
e
a
r
c
h
 
t
e
a
m
 
a
n
d
 
w
i
l
l
 
e
n
g
a
g
e
 
i
n
 
s
u
b
s
t
a
n
t
i
v
e
 
q
u
a
n
t
i
t
a
t
i
v
e
 
r
e
s
e
a
r
c
h
 
a
n
d
a
n
a
l
y
t
i
c
a
l
 
w
o
r
k
 
t
o
 
a
d
v
a
n
c
e
 
a
…
T
h
e
 
D
A
 
h
e
l
p
s
 
p
r
o
v
i
d
e
 
t
i
m
e
l
y
,
 
a
c
c
u
r
a
t
e
 
a
n
d
 
o
b
j
e
c
t
i
v
e
 
a
l
l
-
s
o
u
r
c
e
 
i
n
t
e
l
l
i
g
e
n
c
e
 
a
n
a
l
y
s
i
s
 
o
n
 
n
a
t
i
o
n
a
l
 
s
e
c
u
r
i
t
y
 
a
n
d
 
f
o
r
e
i
g
n
 
p
o
l
i
c
y
 
i
s
s
u
e
s
 
t
o
 
s
e
n
i
o
r
 
p
o
l
i
c
y
m
a
k
e
r
s
 
i
n
…
A
s
 
a
n
 
I
n
t
e
l
l
i
g
e
n
c
e
 
A
n
a
l
y
s
t
 
I
n
t
e
r
n
 
f
o
r
 
t
h
e
 
C
I
A
,
 
y
o
u
 
w
i
l
l
 
w
o
r
k
 
o
n
 
t
e
a
m
s
 
a
l
o
n
g
s
i
d
e
 
o
u
r
 
f
u
l
l
-
t
i
m
e
 
a
n
a
l
y
s
t
s
,
 
s
t
u
d
y
i
n
g
 
a
n
d
 
e
v
a
l
u
a
t
i
n
g
 
i
n
f
o
r
m
a
t
i
o
n
 
f
r
o
m
 
a
l
l
 
a
v
a
i
l
a
b
l
e
…
A
p
p
l
y
 
n
e
w
 
t
e
c
h
n
i
q
u
e
s
 
a
n
d
 
d
e
v
e
l
o
p
 
c
r
e
a
t
i
v
e
 
s
o
l
u
t
i
o
n
s
 
t
o


In [21]:
with open('file.txt') as duties_file:
    print(duties_file.read().split())

['As', 'a', 'Political', 'Analyst', 'for', 'the', 'CIA,', 'you', 'will', 'support', 'policymakers', 'by', 'producing', 'and', 'delivering', 'written', 'and', 'oral', 'assessments', 'of', 'the', 'domestic', 'politics,', 'foreign…Mentoring,', 'coaching,', 'and', 'leading', 'other', 'Language', 'Analysts;', 'In', 'most', 'cases,', 'Experienced', 'Level', 'Language', 'Analysts', 'at', 'NSA', 'will', 'have', 'substantial', 'experience', 'in', 'one', 'or…Our', 'research', 'analysts', 'produce', 'detailed,', 'comprehensive,', 'and', '100%', 'accurate', 'reports', 'on', 'the', 'individuals', '(public', 'figures,', 'activists,', 'legislators,', 'political…The', 'Situation', 'Room,', 'anchored', 'by', 'Wolf', 'Blitzer,', 'assembles', 'top', 'CNN', 'correspondents,', 'analysts,', 'contributors', 'and', 'guests', 'for', 'complete,', 'up-to-the', 'minute', 'coverage', 'of', 'the…Policy', 'analysis,', 'data', 'science,', 'political', 'science,', 'legislative', 'or', 'executive', 'agency', 'policymak

In [22]:
from collections import Counter

In [25]:
import xlwt

book=xlwt.Workbook()
sheet_test=book.add_sheet('word_count')
i = 0
sheet_test.write(i,0,'word')
sheet_test.write(i,1,'count')
with open('file.txt') as duties_file:
    count_result = Counter(duties_file.read().split())
    for result in count_result.most_common(10):
        i = i+1
        sheet_test.write(i,0,result[0])
        sheet_test.write(i,1,result[1])
book.save('result.xls')

In [26]:
%%html
<img src = 'wordle.png'>